In [1]:
import pickle, pprint
import math
import netCDF4
import numpy as np
import time
import matplotlib.pyplot as plt
from scipy import stats
from scipy.ndimage import gaussian_filter
import pandas as pd
from tabulate import tabulate
import xarray as xr
import dask

import seaborn as sns

import matplotlib
from matplotlib.gridspec import GridSpec
from matplotlib import animation
from matplotlib import transforms
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.image as imag
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as mticker
import types


import matplotlib.colors as colors
import matplotlib.patches as patches
from matplotlib import ticker
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs

import csv
import pandas as pd
import os

from mpl_toolkits.basemap import Basemap
bm = Basemap()

import types
def bottom_offset(self, bboxes, bboxes2):
    bottom = self.axes.bbox.ymin
    self.offsetText.set(va="top", ha="left")
    self.offsetText.set_position(
            (0, bottom - self.OFFSETTEXTPAD * self.figure.dpi / 72.0))


def register_bottom_offset(axis, func):
    axis._update_offset_text_position = types.MethodType(func, axis)

/export/home/gmooers/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.


In [2]:
path_to_file = '/fast/gmooers/Raw_Data/extras/TimestepOutput_Neuralnet_SPCAM_216.cam.h0.2018-07.nc'
land_frac = xr.open_dataset(path_to_file)
land_f = np.squeeze(land_frac.LANDFRAC.values)
print(land_f.shape)

(96, 144)


In [3]:
fz = 15*1.5
lw = 4
siz = 100
XNNA = 1.25 # Abscissa where architecture-constrained network will be placed
XTEXT = 0.25 # Text placement
YTEXT = 0.3 # Text placement

plt.rc('text', usetex=False)
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
#mpl.rcParams["font.serif"] = "STIX"
plt.rc('font', family='serif', size=fz)
matplotlib.rcParams['lines.linewidth'] = lw

In [4]:
path_to_file = '../Preprocessed_Data/Summer_Months/full_physics_essentials_valid_month02_targets.nc'
real_ds = xr.open_dataset(path_to_file)

path_to_file = '../Models/Summer_Sigmoid_Summer.nc'
Softmax_ds = xr.open_dataset(path_to_file)

path_to_file = '../Models/Summer_Softplus_Summer.nc'
Softplus_ds = xr.open_dataset(path_to_file)

path_to_file = '../Models/Summer_Exponential_Summer.nc'
Exponential_ds = xr.open_dataset(path_to_file)

path_to_file = '../Models/3_Months_Summer_Linear_Baseline.nc'
Linear_Baseline_ds = xr.open_dataset(path_to_file)

#path_to_file = 'Models/Paper_Full_2018.nc'
#prehttp://localhost:8899/notebooks/Real_Geography_Manuscript/Mooers_Logbook/Precip_PDF_Figure.ipynb#dict_ds = xr.open_dataset(path_to_file)

FileNotFoundError: [Errno 2] No such file or directory: b'/fast/gmooers/Real_Geography_Manuscript/Models/3_Months_Summer_Linear_Baseline.nc'

In [26]:
precip_SPCAM5_ds = np.squeeze(real_ds.targets[:,-1].values)
precip_Softmax_ds = np.squeeze(Softmax_ds.Prediction[:,-1].values)
precip_Softplus_ds = np.squeeze(Softplus_ds.Prediction[:,-1].values)
precip_Exponential_ds = np.squeeze(Exponential_ds.Prediction[:,-1].values)
precip_Linear_Baseline_ds = np.squeeze(Linear_Baseline_ds.Prediction[:,-1].values)

precip_SPCAM5_ds = precip_SPCAM5_ds*86400000.
precip_Softmax_ds = precip_Softmax_ds*86400000.
precip_Softplus_ds = precip_Softplus_ds*86400000.
precip_Exponential_ds = precip_Exponential_ds*86400000.
precip_Linear_Baseline_ds = precip_Linear_Baseline_ds*86400000.

times = real_ds.time.values

lats = np.linspace(-90.0, 90.0, 96)
lons = np.linspace(0.0, 357.5, 144)

Xvar, Yvar = np.meshgrid(lons, lats)

x = 144
y = 96
z = 30
t = int(len(precip_SPCAM5_ds)/(x*y))
print(t)

8831


In [27]:
Softmax_all = precip_Softmax_ds.ravel()
Softplus_all = precip_Softplus_ds.ravel()
Exponential_all = precip_Exponential_ds.ravel()
Linear_Baseline_all = precip_Linear_Baseline_ds.ravel()

all_max_values = [np.nanmax(Softmax_all), np.nanmax(Softplus_all), np.nanmax(Exponential_all), np.nanmax(Linear_Baseline_all)]
all_min_values = [np.nanmin(Softmax_all), np.nanmin(Softplus_all), np.nanmin(Exponential_all), np.nanmin(Linear_Baseline_all)]
m_ocean = min(all_min_values)
M_ocean = max(all_max_values)

In [28]:
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Paper_Summer_SPCAM5_Precip_Sigmoid_PDF.npy",Softmax_all)
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Paper_Summer_SPCAM5_Precip_Softplus_PDF.npy",Softplus_all)
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Paper_Summer_SPCAM5_Prepip_Exponential_PDF.npy",Exponential_all)
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Paper_Summer_SPCAM5_Precip_Linear_Baseline_PDF.npy",Linear_Baseline_all)


In [29]:
precip_SPCAM5 = np.empty(shape=(t, y, x))
precip_Softmax = np.empty(shape=(t, y, x))
precip_Softplus = np.empty(shape=(t, y, x))
precip_Exponential = np.empty(shape=(t, y, x))
precip_Linear_Baseline = np.empty(shape=(t, y, x))
count = 0
for i in range(t):
    for j in range(y):
        for k in range(x):
            precip_SPCAM5[i, j, k] = precip_SPCAM5_ds[count]
            precip_Softmax[i, j, k] = precip_Softmax_ds[count]
            precip_Softplus[i, j, k] = precip_Softplus_ds[count]
            precip_Exponential[i, j, k] = precip_Exponential_ds[count]
            precip_Linear_Baseline[i, j, k] = precip_Linear_Baseline_ds[count]
            
            count = count + 1

In [30]:
precip_SPCAM5_land = np.empty(shape=(t, y, x))
precip_Softmax_land = np.empty(shape=(t, y, x))
precip_Softplus_land = np.empty(shape=(t, y, x))
precip_Exponential_land = np.empty(shape=(t, y, x))
precip_Linear_Baseline_land = np.empty(shape=(t, y, x))

precip_SPCAM5_ocean = np.empty(shape=(t, y, x))
precip_Softmax_ocean = np.empty(shape=(t, y, x))
precip_Softplus_ocean = np.empty(shape=(t, y, x))
precip_Exponential_ocean = np.empty(shape=(t, y, x))
precip_Linear_Baseline_ocean = np.empty(shape=(t, y, x))

In [31]:
for i in range(len(precip_SPCAM5[0])):
    for j in range(len(precip_SPCAM5[0][0])):
        if land_f[i,j] >= 0.5:
            precip_SPCAM5_land[:, i, j] = precip_SPCAM5[:, i, j]
            precip_Softmax_land[:, i, j] = precip_Softmax[:, i, j]
            precip_Softplus_land[:, i, j] = precip_Softplus[:, i, j]
            precip_Exponential_land[:, i, j] = precip_Exponential[:, i, j]
            precip_Linear_Baseline_land[:, i, j] = precip_Linear_Baseline[:, i, j]
        if land_f[i,j] < 0.5:
            precip_SPCAM5_ocean[:, i, j] = precip_SPCAM5[:, i, j]
            precip_Softmax_ocean[:, i, j] = precip_Softmax[:, i, j]
            precip_Softplus_ocean[:, i, j] = precip_Softplus[:, i, j]
            precip_Exponential_ocean[:, i, j] = precip_Exponential[:, i, j]
            precip_Linear_Baseline_ocean[:, i, j] = precip_Linear_Baseline[:, i, j]

In [32]:
SPCAM5_ocean = precip_SPCAM5_ocean.ravel()
SPCAM5_land = precip_SPCAM5_land.ravel()

Softmax_ocean = precip_Softmax_ocean.ravel()
Softmax_land = precip_Softmax_land.ravel()

Softplus_ocean = precip_Softplus_ocean.ravel()
Softplus_land = precip_Softplus_land.ravel()

Exponential_ocean = precip_Exponential_ocean.ravel()
Exponential_land = precip_Exponential_land.ravel()

Linear_Baseline_ocean = precip_Linear_Baseline_ocean.ravel()
Linear_Baseline_land = precip_Linear_Baseline_land.ravel()

In [34]:
SPCAM5_ocean = SPCAM5_ocean[~np.isnan(SPCAM5_ocean)]
SPCAM5_land = SPCAM5_land[~np.isnan(SPCAM5_land)]

Softmax_ocean = Softmax_ocean[~np.isnan(Softmax_ocean)]
Softmax_land = Softmax_land[~np.isnan(Softmax_land)]

Softplus_ocean = Softplus_ocean[~np.isnan(Softplus_ocean)]
Softplus_land = Softplus_land[~np.isnan(Softplus_land)]

Exponential_ocean = Exponential_ocean[~np.isnan(Exponential_ocean)]
Exponential_land = Exponential_land[~np.isnan(Exponential_land)]

Linear_Baseline_ocean = Linear_Baseline_ocean[~np.isnan(Linear_Baseline_ocean)]
Linear_Baseline_land = Linear_Baseline_land[~np.isnan(Linear_Baseline_land)]

In [35]:
ocean_max_values = [np.nanmax(SPCAM5_ocean), np.nanmax(Softmax_ocean), np.nanmax(Softplus_ocean), np.nanmax(Exponential_ocean), np.nanmax(Linear_Baseline_ocean)]
ocean_min_values = [np.nanmin(SPCAM5_ocean), np.nanmin(Softmax_ocean), np.nanmin(Softplus_ocean), np.nanmin(Exponential_ocean), np.nanmin(Linear_Baseline_ocean)]
m_ocean = min(ocean_min_values)
M_ocean = max(ocean_max_values)
land_max_values = [np.nanmax(SPCAM5_land), np.nanmax(Softmax_land), np.nanmax(Softplus_land), np.nanmax(Exponential_land), np.nanmax(Linear_Baseline_land)]
land_min_values = [np.nanmin(SPCAM5_land), np.nanmin(Softmax_land), np.nanmin(Softplus_land), np.nanmin(Exponential_land), np.nanmin(Linear_Baseline_land)]
m_land = min(land_min_values)
M_land = max(land_max_values)

In [36]:
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Paper_Summer_Ocean_SPCAM5_Precip_Sigmoid_PDF.npy",Softmax_ocean)
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Paper_Summer_Ocean_SPCAM5_Precip_Softplus_PDF.npy",Softplus_ocean)
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Paper_Summer_Ocean_SPCAM5_Prepip_Exponential_PDF.npy",Exponential_ocean)
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Paper_Summer_Ocean_SPCAM5_Precip_Linear_Baseline_PDF.npy",Linear_Baseline_ocean)

np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Paper_Summer_Land_SPCAM5_Precip_Sigmoid_PDF.npy",Softmax_land)
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Paper_Summer_Land_SPCAM5_Precip_Softplus_PDF.npy",Softplus_land)
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Paper_Summer_Land_SPCAM5_Prepip_Exponential_PDF.npy",Exponential_land)
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Paper_Summer_Land_CSPAM5_Precip_Linear_Baseline_PDF.npy",Linear_Baseline_land)


print("precip data saved")

precip data saved
